# 基于协同过滤的临床疾病辅助诊断方法研究

## 1.数据预处理

- 导入数据，数据清洗
- 数据变换，连续属性离散化
- 数据可视化

### 1.1 导入数据，数据清洗

In [1]:
# -*- coding:utf-8 -*-
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt

In [55]:
def load_dermatology_dataset(filename):
    data=pd.read_csv(filename,delimiter=',')
    if 'mice'in filename or 'KNN'in filename:
        data.rename(columns={ data.columns[0]: "delete" }, inplace=True)
        data.drop('delete',axis=1,inplace=True)
    target_names_nums={'psoriasis':112,'seboreic dermatitis':61,'lichen planus':72,'pityriasis rosea':49,'cronic dermatitis':52,'pityriasis rubra pilaris':20}
    target_names=['psoriasis','seboreic dermatitis','lichen planus', 'pityriasis rosea','cronic dermatitis','pityriasis rubra pilaris']
    target_nums=[112,61,72,49,52,20]
    target_labels=[]
    if data.shape[1]==34:
        y_num=pd.read_csv('/home/min/bigdata/data/ex1/data/dermatologyNumric.csv',delimiter=',')['target-label-num']
        data=data.join(y_num)
    y_num=data['target-label-num']
    data.rename(columns={ data.columns[34]: "class" }, inplace=True)
    data['class']=data['class']-1
    for c in y_num:
        target_labels.append(target_names[int(c-1)])
    plot_data=data.copy()
    y_class=pd.DataFrame(target_labels,columns=['class'])
    plot_data['class']=y_class
    return plot_data,data

### 1.2 数据变换，连续属性离散化变成分类属性

In [16]:
def get_category(data,k=6):
    ages_with_nan=data.iloc[:,33]
    ages=sorted(ages_with_nan.dropna().values)
    rate_list=[]
    for i in range(1,k):
        rate_list.append(i/float(k))
    rate=np.array(rate_list)
    qs=len(ages)*rate
    qs1=np.ceil(qs)
    qs2=np.floor(qs)
    qs1=qs1.astype(np.int32)
    qs2=qs2.astype(np.int32)
    qss=[]
    for i in range(len(rate)):
        qss.append((ages[qs1[i]]+ages[qs2[i]])/2)
    def change2category(x):
        if not np.isnan(x):
            c=0
            while(c<len(qss) and x>qss[c]):
                c+=1
            return c
        return x
    ages_c=map(change2category,ages_with_nan.values)
    new_ages=pd.Series(ages_c)
    data.iloc[:,33]=new_ages
    return data

### 1.3 数据可视化

In [4]:
from pandas.tools.plotting import parallel_coordinates
def plot_data(plot_data):
    plt.figure(figsize=(15,8))
    parallel_coordinates(plot_data,'target-label')
    plt.show()

### 1.4 处理类别不平衡状态

In [5]:
from smote import Smote
def get_smote_sampledata(data):
    x=data.iloc[:,:-1].values
    y=data.iloc[:,-1].values #数字类别
    target_nums=[112,61,72,49,52,20]
    mx=target_nums[0]
    x_sample=x[y==1]
    y_sample=np.ones((len(x_sample),1))
    for i in range(1,6):
        srate=int(np.floor(mx/target_nums[i]))
        print srate
        st=Smote(sampling_rate=srate, k=6)
        xo=(st.fit(x[y==(i+1)]))
        x_sample=np.concatenate((x_sample,xo), axis=0)
        y_sample=np.concatenate((y_sample,(i+1)*np.ones((len(xo),1))), axis=0)
    data_sample=np.column_stack((x_sample,y_sample))
    return x_sample,y_sample

## 2. 实验

In [53]:
def create_discrete_dir_filenames(path):
    for fname in os.listdir(path):
        if  'dermatology'in fname and  'Mean'in fname and fname.endswith('.csv'):
            print fname
            _,data=load_dermatology_dataset(path+'/'+fname)
            data=get_category(data)
            data.to_csv(path+'/discrete_data/dermatology/discrete_Numric'+fname,index=False,sep=',')
        elif  'dermatology'in fname and  'Numric'in fname and fname.endswith('.csv'):
            print fname
            _,data=load_dermatology_dataset(path+'/'+fname)
            data=get_category(data)
            data.to_csv(path+'/discrete_data/dermatology/discrete_'+fname,index=False,sep=',')
    print 'Job done...'

In [56]:
create_discrete_dir_filenames('/home/min/bigdata/data/ex1/data')

mice_cart_dermatologyNumric.csv
mice_pmm_dermatology_missing30Numric.csv
KNN1_dermatology_missing10Numric.csv
dermatologyMean.csv
mice_pmm_dermatologyNumric.csv
dermatologyNumric.csv
dermatology_missing20Numric.csv
dermatology_missing30Mean.csv
mice_cart_dermatology_missing10Numric.csv
mice_cart_dermatology_missing20Numric.csv
KNN1_dermatologyNumric.csv
mice_rf_dermatology_missing20Numric.csv
mice_cart_dermatology_missing30Numric.csv
dermatology_missing10Mean.csv
mice_rf_dermatology_missing30Numric.csv
dermatology_missing10Numric.csv
mice_pmm_dermatology_missing20Numric.csv
mice_pmm_dermatology_missing10Numric.csv
dermatology_missing20Mean.csv
dermatology_missing30Numric.csv
mice_rf_dermatologyNumric.csv
mice_rf_dermatology_missing10Numric.csv
Job done...


## 普通的机器学习方法 

In [15]:
# 从sklearn.cross_validation中导入train_test_split用于数据分割。
from sklearn.cross_validation import train_test_split

# 从sklearn.preprocessing里导入数据标准化模块。
from sklearn.preprocessing import StandardScaler

# 从sklearn.metrics里导入classification_report模块。
from sklearn.metrics import classification_report

# 从sklearn.linear_model里导入LogisticRegression与SGDClassifier。
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier

# 从sklearn.svm里导入基于线性假设的支持向量机分类器LinearSVC。
from sklearn.svm import LinearSVC

#多层感知器MLP
from sklearn.neural_network import MLPClassifier

# 对类别型特征进行转化，成为特征向量。
from sklearn.feature_extraction import DictVectorizer

# 使用随机森林分类器进行集成模型的训练以及预测分析。
from sklearn.ensemble import RandomForestClassifier

import pandas as pd
import numpy as np
"""
target_names_nums={'psoriasis':112,'seboreic dermatitis':61,'lichen planus':72,'pityriasis rosea':49,'cronic dermatitis':52,'pityriasis rubra pilaris':20}
target_names=['psoriasis','seboreic dermatitis','lichen planus', 'pityriasis rosea','cronic dermatitis','pityriasis rubra pilaris']
target_nums=[112,61,72,49,52,20]
"""

path='/home/min/bigdata/data/ex1/data'
fname='mice_pmm_chronic_missing10Numric.csv'
def get_precision_f1(report):
    lines=report.split('\n')
    line=lines[len(lines)-2]
    row_data = filter(None,line.split(' '))
    precision=float(row_data[3])
    f1=float(row_data[5])
    return (precision,f1)
#LR SVM MLP RF
def report_for_machinelearn(path,fname):
    precisons={}
    f1s={}
    data=pd.read_csv(path+'/'+fname,delimiter=',')
    if 'mice'in fname or 'KNN'in fname:
        data.rename(columns={ data.columns[0]: "delete" }, inplace=True)
        data.drop('delete',axis=1,inplace=True)
    
    X=data.iloc[:,:data.shape[1]-1]
    Y=data.iloc[:,data.shape[1]-1]

    # 随机采样20%的数据用于测试，剩下的80%用于构建训练集合。
    X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.2, random_state=33)
    #print pd.value_counts(y_train.values.flatten())
    #print pd.value_counts(y_test.values.flatten())


    # 初始化LogisticRegression与SGDClassifier。
    lr = LogisticRegression()
    sgdc = SGDClassifier()

    # 调用LogisticRegression中的fit函数/模块用来训练模型参数。
    lr.fit(X_train, y_train)
    # 使用训练好的模型lr对X_test进行预测，结果储存在变量lr_y_predict中。
    lr_y_predict = lr.predict(X_test)

    # 使用逻辑斯蒂回归模型自带的评分函数score获得模型在测试集上的准确性结果。
    print 'Accuracy of LR Classifier:', lr.score(X_test, y_test)
    # 利用classification_report模块获得LogisticRegression其他三个指标的结果。
    report=classification_report(y_test, lr_y_predict,digits = 5)
    rets=(get_precision_f1(report))
    precisons['LR']=rets[0]
    f1s['LR']=rets[1]



    # 初始化线性假设的支持向量机分类器LinearSVC。
    lsvc = LinearSVC()
    #进行模型训练
    lsvc.fit(X_train, y_train)
    # 利用训练好的模型对测试样本的数字类别进行预测，预测结果储存在变量y_predict中。
    y_predict = lsvc.predict(X_test)
    # 使用模型自带的评估函数进行准确性测评。
    print 'The Accuracy of SVM is', lsvc.score(X_test, y_test)

    report=classification_report(y_test,y_predict,digits = 5)
    rets=(get_precision_f1(report))
    precisons['SVM']=rets[0]
    f1s['SVM']=rets[1]



    # 初始化多层感知器MLP
    mlp = MLPClassifier(solver='lbfgs', alpha=1e-5,  
                        hidden_layer_sizes=(6, 6), random_state=1)
    mlp.fit(X_train, y_train)
    mlp_y_pred=mlp.predict(X_test) 

    # 输出多层感知分类器在测试集上的分类准确性，以及更加详细的精确率、召回率、F1指标。
    print 'The accuracy of MLP Classifier is', mlp.score(X_test, y_test)
    report=classification_report(y_test,mlp_y_pred,digits = 5)
    rets=get_precision_f1(report)
    precisons['MLP']=rets[0]
    f1s['MLP']=rets[1]


    # 使用随机森林分类器RF
    # 对类别型特征进行转化，成为特征向量。
    vec = DictVectorizer(sparse=False)
    X_train = vec.fit_transform(X_train.to_dict(orient='record'))
    X_test = vec.transform(X_test.to_dict(orient='record'))

    rfc = RandomForestClassifier()
    rfc.fit(X_train, y_train)
    rfc_y_pred = rfc.predict(X_test)

    # 输出随机森林分类器在测试集上的分类准确性，以及更加详细的精确率、召回率、F1指标。
    print 'The accuracy of random forest classifier is', rfc.score(X_test, y_test)
    report=classification_report(y_test,rfc_y_pred,digits = 5)
    rets=get_precision_f1(report)
    precisons['RF']=rets[0]
    f1s['RF']=rets[1]
    return precisons,f1s
print report_for_machinelearn(path,fname)

Accuracy of LR Classifier: 0.962962962963
The Accuracy of SVM is 0.938271604938
The accuracy of MLP Classifier is 0.901234567901
The accuracy of random forest classifier is 0.913580246914
({'MLP': 0.81222, 'SVM': 0.94223, 'LR': 0.96153, 'RF': 0.89775}, {'MLP': 0.85442, 'SVM': 0.92526, 'LR': 0.96185, 'RF': 0.89537})


In [6]:
print classification_report(lr_y_predict,y_test,digits = 5)

             precision    recall  f1-score   support

          0    1.00000   0.96053   0.97987        76
          1    0.62500   1.00000   0.76923         5

avg / total    0.97685   0.96296   0.96686        81



## 一个目录中多个文件运行传统机器学习方法得到的实验结果

In [26]:
import os
data_path='/home/min/bigdata/data/ex1/data'
fnames=[]
f1s={}
precisons={}
for fname in os.listdir(data_path):
    if 'Numric' in fname and 'KNN' not in fname and ('Mean' in fname or 'KNN' in fname or 'mice' in fname):
        print fname
        pre,f1=report_for_machinelearn(path,fname)
        f1s[fname]=f1
        precisons[fname]=pre
        
precisions_df= pd.DataFrame.from_dict(precisons)
precisions_df=precisions_df.T

precisions_df.to_csv('/home/min/bigdata/data/ex1/ml_classification_precisions.csv', index = True)
f1_scores_df = pd.DataFrame.from_dict(f1s)
f1_scores_df=f1_scores_df.T
f1_scores_df.to_csv('/home/min/bigdata/data/ex1/ml_classification_f1_score.csv', index = True)
print f1_scores_df.head(5)
print precisions_df.head(5)

mice_pmm_chronic_missing10Numric.csv
Accuracy of LR Classifier: 0.962962962963
The Accuracy of SVM is 0.925925925926
The accuracy of MLP Classifier is 0.901234567901
The accuracy of random forest classifier is 0.938271604938
mice_cart_dermatologyNumric.csv
Accuracy of LR Classifier: 0.0
The Accuracy of SVM is 0.0135135135135
The accuracy of MLP Classifier is 0.0135135135135
The accuracy of random forest classifier is 0.027027027027
mice_pmm_dermatology_missing30Numric.csv
Accuracy of LR Classifier: 0.0810810810811
The Accuracy of SVM is 0.0540540540541
The accuracy of MLP Classifier is 0.0675675675676
The accuracy of random forest classifier is 0.0540540540541
mice_pmm_chronic_missing20Numric.csv
Accuracy of LR Classifier: 0.975308641975
The Accuracy of SVM is 0.16049382716
The accuracy of MLP Classifier is 0.925925925926
The accuracy of random forest classifier is 0.962962962963
mice_rf_chronic_missing20Numric.csv
Accuracy of LR Classifier: 0.925925925926
The Accuracy of SVM is 0.9629

                                      LR      MLP       RF      SVM
chronicNumricMean.csv            0.96295  0.61077  0.97531  0.51640
chronic_missing10NumricMean.csv  0.95062  0.65135  0.97530  0.82143
chronic_missing20NumricMean.csv  0.95057  0.81481  0.92579  0.67375
chronic_missing30NumricMean.csv  0.96295  0.32650  0.95053  0.81481
mice_cart_chronicNumric.csv      0.94305  0.85442  0.96185  0.04232
                                      LR      MLP       RF      SVM
chronicNumricMean.csv            0.96323  0.62794  0.97531  0.68222
chronic_missing10NumricMean.csv  0.95062  0.66143  0.97648  0.87116
chronic_missing20NumricMean.csv  0.95169  0.81506  0.93020  0.81311
chronic_missing30NumricMean.csv  0.96323  0.24387  0.95511  0.81506
mice_cart_chronicNumric.csv      0.95318  0.81222  0.96153  0.91111
